### Automation Script Overhaul 
For automating the parsing, transformation, and geographic file creation from .GPX<br><br>
Work taken from the script by Isias (ler_gpx.py) and workflow by Simone. Further developments by Kyle & Isais. <br><br>
Data collection must be done with Locus Map 4.x or formatted similarly to GPX files exportd by Locus Map 4 for this script to work.

### Section 1
This section takes the raw GPX files and makes them ready for QGIS and statistical analysis.

In [ ]:
import gpxpy
import pandas as pd
import geopandas as gpd
import os
from glob import glob

In [ ]:
gpxDict = dict()

for file in os.listdir():
    if file.endswith('.gpx'):
       gpxDict[file] = 'file_'+file
gpxDict

In [ ]:
# gpxList = list(gpxDict)
# gpxList

In [ ]:
# d = {} #create empty dictionary
merged = pd.DataFrame(columns=['name','lat','lon','ele'])


for i in gpxDict:
    gpxCurrent = i
    gpxCurrent = open(gpxCurrent)
    gpxCurrent = gpxpy.parse(gpxCurrent)
    gpxCurrent = gpxCurrent.to_xml()
    df = pd.read_xml(gpxCurrent) #open and read in the .gpx to a dataframe
    df.pop('desc')
    df.pop('hdop')
    df.pop('time')#remove unecessary columns
    df = df.drop(index=0)
    shiftPos = df.pop('name')
    df.insert(0, 'name', shiftPos)#reorganize columns
    # print(df)
    df.to_csv('csv', sep=',', index=False)
    # d[i] = pd.DataFrame #create dict of datafrmaes created using file names
# d
merged

In [ ]:
# d = {} #create empty dictionary
merged = pd.DataFrame(columns=['name','lat','lon','ele'])


for i in gpxDict:
    gpxCurrent = i
    gpxCurrent = open(gpxCurrent)
    gpxCurrent = gpxpy.parse(gpxCurrent)
    gpxCurrent = gpxCurrent.to_xml()
    df = pd.read_xml(gpxCurrent) #open and read in the .gpx to a dataframe
    df.pop('desc')
    df.pop('hdop')
    df.pop('time')#remove unecessary columns
    df = df.drop(index=0)
    shiftPos = df.pop('name')
    df.insert(0, 'name', shiftPos)#reorganize columns)
    print(df)
    # merged = pd.concat()
    # d[i] = pd.DataFrame #create dict of datafrmaes created using file names
# d

In [ ]:
d.type()

In [ ]:
df = pd.concat(d.values(), ignore_index=True)
df

In [ ]:
for file in os.listdir('GPX_Exports'):
    if file.endswith('.gpx'):
        gpxCurrent = file
        gpxCurrent = open(gpxCurrent)
        gpxCurrent = gpxpy.parse(gpxCurrent)
        gpxCurrent = gpxCurrent.to_xml()
        # gpxCurrent.head()

In [ ]:
#import dependiencies
import gpxpy
import pandas as pd
import geopandas as gpd
import os
from glob import glob

# see all GPX files from a saved directory
gpxFiles = glob('GPX_Exports/*.gpx')
gpxFiles.sort()
gpxFiles

##### Choose the cell below for single file manipulation OR the cell two below for mass import and manipulation.

Single File Analysis<br>
V<br>
V

In [ ]:
for item in gpxFiles:
    gpxCurrent = gpxFiles[0]
    gpxCurrent = open(gpxCurrent)
    gpxCurrent = gpxpy.parse(gpxCurrent)
    gpxCurrent = gpxCurrent.to_xml()
    gpxCurrent.head()

In [ ]:
# Choose which file to use, select (using 0-n) the gpx file to manipulate / Escolha qual arquivo usar, selecione (usando 0-n) o arquivo gpx para manipular
gpxCurrent = gpxFiles[0]
gpxCurrent = open(gpxCurrent)
gpxCurrent = gpxpy.parse(gpxCurrent)
gpxCurrent = gpxCurrent.to_xml()

# Read the XML into a dataframe, remove unnecessary columns, and shift columns / Leia o XML em um dataframe, remova colunas desnecessárias e desloque colunas
df = pd.read_xml(gpxCurrent)
df.pop('desc')
df.pop('hdop')
df.pop('time')
df = df.drop(index=0)
shiftPos = df.pop('name')
df.insert(0, 'name', shiftPos)
df.head()

In [ ]:
# Merge all GPX files in directory / Mesclar todos os arquivos GPX no diretório
for i in gpxDict:
    df = pd.read_xml(file)
    df.pop('desc')
    df.pop('hdop')
    df.pop('time')
    df = df.drop(index=0)
    shiftPos = df.pop('name')
    df.insert(0, 'name', shiftPos)

df = pd.concat(gpxFiles,ignore_index=True)

df.head()

In [ ]:
# Convert to geodataframe for geographic use / Converter para geodataframe para uso geográfico
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.lon, df.lat)) # if Z is wanted for the points, add / se Z for desejado para os pontos, adicione ', df.ele' ex. (df.lon, df.lat, df.ele)
gdf = gdf.set_crs('EPSG:4326')
gdf.head()

In [ ]:
# Ask for observer, group, climate conditions / Pergunte por observador, grupo, condições climáticas
observer = input('Observer/Observador? ')
group = input('Group/Grupo? (if both, mark 0) ') # or leave blank?
weather = input('Weather conditions/Condição do clima? ')
print('Observer/Observador: '+observer)
print('Group/Grupo: '+group)
print('Weather/Tempo: '+weather)

 If the above is correct, continue here. If not, re-run the cell and correct the information<br><br>
 Se o acima estiver correto, continue aqui. Caso contrário, execute novamente a célula e corrija as informações

In [ ]:
# Add the input to the dataframe
gdf.insert(loc=1, column='observer', value=observer, allow_duplicates=True)
gdf.insert(loc=1, column='group', value=group, allow_duplicates=True)
gdf.insert(loc=1, column='weather', value=weather, allow_duplicates=True)
gdf.head()

In [ ]:
# Split 'name' into date, time, and observations / Dividir 'nome' em data, hora e observações
date = gdf['name'].str[:10]
gdf.insert(loc=0, column='date', value=date, allow_duplicates=True)

time = gdf['name'].str[11:19]
gdf.insert(loc=1, column='time', value=time, allow_duplicates=True)

obs = gdf['name'].str[19:]
gdf.insert(loc=2, column='observations', value=obs, allow_duplicates=True)

gdf.pop('name')

gdf.head()

In [ ]:
# # split observations into Age/Sex, Strata position, and behaviour
# ageSex = ''
# strata = ''
# behaviour = ''
# gdf.insert(loc=2, column='age/sex', value=ageSex, allow_duplicates=True)
# gdf.insert(loc=3, column='strata', value=strata, allow_duplicates=True)
# gdf.insert(loc=4, column='behaviour', value=behaviour, allow_duplicates=True)

In [ ]:
# Run loop to identify age, sex, and behaviour
for i, row in gdf.iterrows():
        # asSubstring = gdf['observations'].str[:2]
        if gdf['observations'].str[:2] == 'j2':
                ageSex = 'Juvenile 2'
                strata = gdf['observations'].str[2]
                behaviour = gdf['observations'].str[3:]
                gdf['age/sex']=ageSex
                gdf['strata']=strata
                gdf['behaviour']=behaviour
                

gdf.head()

In [ ]:
gdf.head()

In [ ]:
# Attempt to detect each scan, else create column for scan number / Tente detectar cada varredura, senão crie uma coluna para o número da varredura


#### Section 2
Analysis of scans, analyze all of this for every individual scan

In [ ]:
# Find centroid of each scan (collect geometries, find centroid)


In [ ]:
# Distance of each point/animal to centroid


In [ ]:
# points to polygons for area (ha) of group spread (convex hull)


In [ ]:
# Distance between each centroid in temporal order


In [ ]:
# Subgroups/cluster analysis, find clusters on eah scan and distance from each sub-centroid to main group centroid


#### Export Data

In [ ]:
# Export data frame to gpkg for use in QGIS / Exportar quadro de dados para gpkg para uso no QGIS
gdf.to_file('gdf.gpkg', driver="GPKG")
gdf.info()

In [ ]:
gdf.plot()